In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import csv
import math
import matplotlib as plt

In [3]:
batch_size=32
csv_file="atlas-higgs-challenge-2014-v2.csv"
crossval_nbr=5


In [4]:
class Dataset_Cust(torch.utils.data.Dataset):
    def __init__(self, csv_file, train=True, crossval_nbr=5, transformation=None):
        'Initialization'
        self.all=np.array(pd.read_csv(csv_file))
        #print(self.all[:,:-3])
        self.data=torch.from_numpy(self.all[:,0:-3].astype('float64'))
        self.train=train
        
        self.crossval_nbr=crossval_nbr
        self.testpart_num=crossval_nbr-1
        
        self.nbrpoint=len(self.all)
        #self.nbrtrain=int((1-1/crossval_nbr)*self.nbrpoint)
        
        self.Taille_section=int(self.nbrpoint/self.crossval_nbr)
        self.debut_test=self.testpart_num * self.Taille_section
        
        if self.testpart_num==self.crossval_nbr-1:
            self.fin_test=self.nbrpoint
        else:
            self.fin_test=self.debut_test + self.Taille_section
        
        self.nbrtest=self.fin_test - self.debut_test
        self.nbrtrain=self.nbrpoint - self.nbrtest
        #self.numtest=self.numpoint - self.numtrain
        
        self.transformation=transformation
        

    def __len__(self):
        'Denotes the total number of samples'
        if self.train:
            return self.nbrtrain
        else:
            return self.fin_test - self.debut_test
    

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        if self.train:
            if index>= self.debut_test:
                index+=self.nbrtest
                
        else:
            index+=self.debut_test
        
        #sample=torch.from_numpy(self.all[index][0:-3].astype('float64'))
        sample=self.data[index]
        sample.requires_grad=True
        label=1 if self.all[index][-3]=="s" else 0
        
        if self.transformation :
            sample=self.transformation(sample)

        return sample.float(), float(label)
    
    def Set_Train(self, train):
        self.train=train
        
    def Set_TestPart_Num(self,num):
        self.testpart_num=num
        
        self.debut_test=self.testpart_num * self.Taille_section
        
        if self.testpart_num==self.crossval_nbr-1:
            self.fin_test=self.nbrpoint
        else:
            self.fin_test=self.debut_test + self.Taille_section
        
        self.nbrtest=self.fin_test - self.debut_test
        self.nbrtrain=self.nbrpoint - self.nbrtest

In [5]:
Dataset=Dataset_Cust(csv_file)

In [6]:
Data_Loader=torch.utils.data.DataLoader(Dataset, batch_size=batch_size, shuffle=True)

In [7]:
def AMS(s,b):
    assert s >= 0
    assert b >= 0
    bReg = 10.
    return math.sqrt(2 * ((s + b + bReg) * 
                          math.log(1 + s / (b + bReg)) - s))


In [8]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron,self).__init__()
        self.fc1=nn.Linear(30,1)
        
    
    def forward(self, x):
        return F.sigmoid(self.fc1(x))

    
class MLP_1(nn.Module):
    def __init__(self):
        super(MLP_1,self).__init__()
        self.fc1=nn.Linear(30,15)
        self.fc2=nn.Linear(15,1)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        return F.sigmoid(self.fc2(x))
        
class MLP_2(nn.Module):
    def __init__(self):
        super(MLP_2,self).__init__()
        self.fc1=nn.Linear(30,15)
        self.fc2=nn.Linear(15,7)
        self.fc3=nn.Linear(7,1)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        return F.sigmoid(self.fc3(x))

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Conv1d(1,10,5)
        self.conv2=nn.Conv1d(10,20,5)
        self.fc=nn.Linear(50*4*4,1)
        
    def forward(self, x):
        x=F.max_pool1d(F.relu(self.conv1(x)),2,2)
        x=F.max_pool1d(F.relu(self.conv2(x)),2,2)
        x=x.view(-1,50*4*4)
        x=self.fc(x)
        return F.sigmoid(x)

In [9]:
def init_weights(m):
    if type(m) == nn.Linear or type(m)==nn.Conv1d:
        torch.nn.init.zeros_(m.weight)
        torch.nn.init.zeros_(m.bias)


In [10]:
Percep=Perceptron()
Mlp1=MLP_1()
Mlp2=MLP_2()
Cnn=CNN()
#print(Mlp1.fc1.weight)

In [11]:
criterion= nn.BCELoss()
optimizerPercep= optim.SGD(Percep.parameters(), lr=0.001, momentum=0.9)
optimizeMlp1= optim.SGD(Mlp1.parameters(), lr=0.001, momentum=0.9)
optimizeMlp2= optim.SGD(Mlp2.parameters(), lr=0.001, momentum=0.9)
optimizeCnn= optim.SGD(Cnn.parameters(), lr=0.001, momentum=0.9)

In [13]:
def Train(Model, Optimizer, Criterion, Data_Loader,Epoch=10, log_interval=100):
    #Data_Loader.dataset.Set_TestPart_Num(4)
    print(Data_Loader.dataset.debut_test)
    predicted=np.empty(batch_size)
    #print(type(Percep.fc1.weight[0][0].item()))
    #print(type(Percep.fc1.bias[0].item()))
    init_weights(Model)
    
    Data_Loader.dataset.train=True
    
    for epoch in range(Epoch):
        ams=0
        TP=0 #True Positive
        FP=0 # False Positive
        S=0 #True Positive pondéré
        B=0 # False Positive pondéré
        Loss=0
        correct=0
    
        fp=0
        tp=0
        total=0
        running_loss=0
        for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            Optimizer.zero_grad()
            
            #print(inputs[0][0].requires_grad)
            #print(labels.size(0))
            #print(inputs[0][0])
            
            
            outputs= Model(inputs[:,1:31])
        
             #print(outputs[0]+outputs[1])
             # print(outputs.view(-1,32))
            
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= Criterion(outputs, labels.float())
            loss.backward()
            Optimizer.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                if tp+fp>0:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                else:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} ".format(epoch+1, batch_id+1, running_loss/log_interval)) 

                total=0
                tp=0
                fp=0
                running_loss=0
        if TP+FP>0:
            print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
        else:
            print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}  , AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), AMS(S,B) ))
        
    print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f},  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))
    return [Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ]
        

In [14]:
#Res=Train(Mlp2, optimizeMlp2, criterion, Data_Loader )
#print(Res)
a=np.arange(4).reshape(2,2)
a[0]=[2,2]
print(a)
print(np.mean(a, axis=0))

[[2 2]
 [2 3]]
[2.  2.5]


In [15]:
def Test(Model, Criterion, Data_Loader):

    correct= 0
    total= 0
    test_loss = 0
    predicted=np.empty(batch_size)
    TP=0
    FP=0
    S=0
    B=0

    Data_Loader.dataset.train=False
    with torch.no_grad():
        for sample in Data_Loader:
            inputs, labels= sample
            #images=images.view(-1,784)
            outputs= Model(inputs[:,1:31])

            total+= labels.size(0)

            for i in range(labels.size(0)):
                    if outputs.data[i]>0.5:
                        predicted[i]=1

                        if labels[i]==1:
                            TP+=1
                            #tp+=1
                            S+= inputs[i][31]
                            correct+=1
                        else:
                            FP+=1
                            #fp+=1
                            B+=inputs[i][31]
                    else:
                        predicted[i]=0
                        if labels[i]==0:
                            correct+=1

            test_loss+=Criterion(outputs,labels.float())




    test_loss/= len(Data_Loader.dataset)

    print("Résultat test: {} bonnes classification sur {} données de test:loss moyen={:.8f} Accuracy={:.3f}%, Precision={:.3f}%, AMS={:.3f}".format(correct,total, test_loss/total,100*correct/total, 100*TP/(TP+FP), AMS(S,B)))
    
    return [test_loss/total, 100*correct/total, 100*TP/(TP+FP), AMS(S,B)]


In [ ]:
######TEST Perceptron
Tableau_Perceptron=np.arange(20).reshape(crossval_nbr,4)
for section in range(crossval_nbr):
    print("Train {} :\n".format(section))
    Data_Loader.dataset.Set_TestPart_Num(section)
    Train(Percep, optimizerPercep, criterion, Data_Loader)
    print("Test {} :\n".format(section))
    Tableau_Perceptron[section]=Test(Percep, criterion, Data_Loader)

Res_Perceptron=mean(Tableau_Perceptron,axis=0)

print("tableau pour les {} étapes de validations:\n".format(crossval_nbr))
print(Tableau_Perceptron)

print("Moyennne des metrics:\n")
print(Res_Perceptron)

Train {} :

0
epoch:1, nombre batch:1 loss:0.060 precision=70.000%


/home/andrei/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch:1, nombre batch:101 loss:10.120 precision=45.697%
epoch:1, nombre batch:201 loss:9.930 precision=44.749%
epoch:1, nombre batch:301 loss:9.576 precision=47.140%
epoch:1, nombre batch:401 loss:9.861 precision=45.307%
epoch:1, nombre batch:501 loss:10.094 precision=46.290%
epoch:1, nombre batch:601 loss:10.059 precision=46.536%
epoch:1, nombre batch:701 loss:9.818 precision=51.522%
epoch:1, nombre batch:801 loss:9.878 precision=44.151%
epoch:1, nombre batch:901 loss:9.688 precision=47.548%
epoch:1, nombre batch:1001 loss:9.757 precision=47.163%
epoch:1, nombre batch:1101 loss:9.524 precision=48.639%
epoch:1, nombre batch:1201 loss:10.077 precision=46.050%
epoch:1, nombre batch:1301 loss:9.334 precision=50.344%
epoch:1, nombre batch:1401 loss:9.818 precision=50.394%
epoch:1, nombre batch:1501 loss:9.818 precision=49.890%
epoch:1, nombre batch:1601 loss:10.128 precision=47.086%
epoch:1, nombre batch:1701 loss:10.198 precision=47.420%
epoch:1, nombre batch:1801 loss:10.137 precision=47

epoch:1, nombre batch:14701 loss:9.774 precision=48.345%
epoch:1, nombre batch:14801 loss:10.379 precision=42.011%
epoch:1, nombre batch:14901 loss:9.800 precision=48.214%
epoch:1, nombre batch:15001 loss:9.662 precision=48.284%
epoch:1, nombre batch:15101 loss:9.705 precision=47.452%
epoch:1, nombre batch:15201 loss:10.016 precision=47.368%
epoch:1, nombre batch:15301 loss:10.293 precision=46.601%
epoch:1, nombre batch:15401 loss:9.818 precision=48.501%
epoch:1, nombre batch:15501 loss:9.913 precision=46.847%
epoch:1, nombre batch:15601 loss:10.180 precision=46.957%
epoch:1, nombre batch:15701 loss:10.223 precision=45.759%
epoch:1, nombre batch:15801 loss:9.982 precision=46.380%
epoch:1, nombre batch:15901 loss:10.275 precision=46.837%
epoch:1, nombre batch:16001 loss:10.059 precision=47.640%
epoch:1, nombre batch:16101 loss:10.379 precision=45.052%
epoch:1, nombre batch:16201 loss:9.913 precision=46.294%
epoch:1, nombre batch:16301 loss:9.913 precision=46.325%
epoch:1, nombre batch:1

/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


...

epoch:4, nombre batch:16901 loss:9.749 precision=48.000%
epoch:4, nombre batch:17001 loss:10.293 precision=45.680%
epoch:4, nombre batch:17101 loss:9.783 precision=47.751%
epoch:4, nombre batch:17201 loss:9.731 precision=48.260%
epoch:4, nombre batch:17301 loss:10.128 precision=46.358%
epoch:4, nombre batch:17401 loss:10.189 precision=46.728%
epoch:4, nombre batch:17501 loss:9.757 precision=47.483%
epoch:4, nombre batch:17601 loss:9.628 precision=46.953%
epoch:4, nombre batch:17701 loss:10.327 precision=43.826%
epoch:4, nombre batch:17801 loss:9.973 precision=45.249%
epoch:4, nombre batch:17901 loss:9.826 precision=45.843%
epoch:4, nombre batch:18001 loss:10.146 precision=45.737%
epoch:4, nombre batch:18101 loss:9.792 precision=49.017%
epoch:4, nombre batch:18201 loss:9.774 precision=47.685%
epoch:4, nombre batch:18301 loss:10.008 precision=47.296%
epoch:4, nombre batch:18401 loss:10.025 precision=48.516%
epoch:4, nombre batch:18501 loss:10.034 precision=45.100%
epoch:4, nombre batch:1

epoch:5, nombre batch:10901 loss:10.085 precision=44.279%
epoch:5, nombre batch:11001 loss:9.887 precision=48.144%
epoch:5, nombre batch:11101 loss:9.861 precision=45.320%
epoch:5, nombre batch:11201 loss:10.534 precision=46.041%
epoch:5, nombre batch:11301 loss:9.619 precision=47.592%
epoch:5, nombre batch:11401 loss:9.895 precision=44.884%
epoch:5, nombre batch:11501 loss:9.679 precision=48.152%
epoch:5, nombre batch:11601 loss:10.180 precision=48.498%
epoch:5, nombre batch:11701 loss:9.662 precision=49.715%
epoch:5, nombre batch:11801 loss:9.921 precision=48.778%
epoch:5, nombre batch:11901 loss:10.258 precision=44.588%
epoch:5, nombre batch:12001 loss:9.835 precision=46.277%
epoch:5, nombre batch:12101 loss:9.731 precision=47.725%
epoch:5, nombre batch:12201 loss:9.697 precision=47.406%
epoch:5, nombre batch:12301 loss:9.662 precision=46.253%
epoch:5, nombre batch:12401 loss:10.068 precision=44.931%
epoch:5, nombre batch:12501 loss:9.844 precision=46.101%
epoch:5, nombre batch:1260

epoch:6, nombre batch:4801 loss:10.085 precision=47.078%
epoch:6, nombre batch:4901 loss:9.766 precision=45.434%
epoch:6, nombre batch:5001 loss:9.939 precision=45.772%
epoch:6, nombre batch:5101 loss:9.990 precision=47.092%
epoch:6, nombre batch:5201 loss:9.826 precision=46.243%
epoch:6, nombre batch:5301 loss:9.835 precision=46.251%
epoch:6, nombre batch:5401 loss:9.800 precision=45.105%
epoch:6, nombre batch:5501 loss:9.610 precision=48.639%
epoch:6, nombre batch:5601 loss:10.483 precision=41.317%
epoch:6, nombre batch:5701 loss:9.947 precision=46.377%
epoch:6, nombre batch:5801 loss:10.111 precision=48.068%
epoch:6, nombre batch:5901 loss:10.137 precision=44.606%
epoch:6, nombre batch:6001 loss:9.844 precision=47.903%
epoch:6, nombre batch:6101 loss:9.964 precision=47.836%
epoch:6, nombre batch:6201 loss:9.783 precision=45.314%
epoch:6, nombre batch:6301 loss:10.068 precision=44.593%
epoch:6, nombre batch:6401 loss:9.800 precision=47.558%
epoch:6, nombre batch:6501 loss:10.085 prec

epoch:6, nombre batch:19201 loss:10.051 precision=45.150%
epoch:6, nombre batch:19301 loss:9.895 precision=48.488%
epoch:6, nombre batch:19401 loss:10.154 precision=46.235%
epoch:6, nombre batch:19501 loss:9.792 precision=45.455%
epoch:6, nombre batch:19601 loss:9.731 precision=47.098%
epoch:6, nombre batch:19701 loss:10.034 precision=44.796%
epoch:6, nombre batch:19801 loss:9.835 precision=48.178%
epoch:6, nombre batch:19901 loss:9.671 precision=46.445%
epoch:6, nombre batch:20001 loss:10.103 precision=46.465%
epoch:6, nombre batch:20101 loss:10.413 precision=46.865%
epoch:6, nombre batch:20201 loss:9.852 precision=47.245%
epoch:6, nombre batch:20301 loss:10.077 precision=46.536%
epoch:6, nombre batch:20401 loss:9.869 precision=45.683%
Resultat Epoch 6;  loss:9.924, Accuracy=64.085  precision:46.815%, AMS=0.6211
epoch:7, nombre batch:1 loss:0.086 precision=33.333%
epoch:7, nombre batch:101 loss:9.878 precision=47.150%
epoch:7, nombre batch:201 loss:10.241 precision=45.295%
epoch:7, no

epoch:7, nombre batch:13201 loss:9.861 precision=47.313%
epoch:7, nombre batch:13301 loss:9.749 precision=46.413%
epoch:7, nombre batch:13401 loss:9.809 precision=46.437%
epoch:7, nombre batch:13501 loss:10.249 precision=45.940%
epoch:7, nombre batch:13601 loss:10.189 precision=46.197%
epoch:7, nombre batch:13701 loss:9.956 precision=47.315%
epoch:7, nombre batch:13801 loss:10.016 precision=47.697%
epoch:7, nombre batch:13901 loss:9.826 precision=45.737%
epoch:7, nombre batch:14001 loss:10.025 precision=45.318%
epoch:7, nombre batch:14101 loss:9.792 precision=49.272%
epoch:7, nombre batch:14201 loss:9.887 precision=45.413%
epoch:7, nombre batch:14301 loss:9.809 precision=47.114%
epoch:7, nombre batch:14401 loss:10.370 precision=46.352%
epoch:7, nombre batch:14501 loss:10.085 precision=46.659%
epoch:7, nombre batch:14601 loss:9.800 precision=44.643%
epoch:7, nombre batch:14701 loss:9.913 precision=45.465%
epoch:7, nombre batch:14801 loss:9.947 precision=45.991%
epoch:7, nombre batch:149

In [16]:
########Test MLP à une Couche Cachée
Tableau_Mlp1=np.arange(20).reshape(crossval_nbr,4)
for section in range(crossval_nbr):
    print("Train {} :\n".format(section))
    Data_Loader.dataset.Set_TestPart_Num(section)
    Train(Mlp1, optimizeMlp1, criterion, Data_Loader)
    print("Test {} :\n".format(section))
    Tableau_Mlp1[section]=Test(Mlp1, criterion, Data_Loader)

Res_Mlp1=mean(Tableau_Mlp1,axis=0)

print("tableau pour les {} étapes de validations:\n".format(crossval_nbr))
print(Tableau_Mlp1)

print("Moyennne des metrics:\n")
print(Res_Mlp1)

Train 0 :

0
epoch:1, nombre batch:1 loss:0.183 


/home/andrei/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch:1, nombre batch:101 loss:17.414 precision=35.963%
epoch:1, nombre batch:201 loss:18.211 precision=34.094%
epoch:1, nombre batch:301 loss:18.185 precision=34.188%
epoch:1, nombre batch:401 loss:18.280 precision=33.844%
epoch:1, nombre batch:501 loss:18.314 precision=33.719%
epoch:1, nombre batch:601 loss:17.848 precision=35.406%
epoch:1, nombre batch:701 loss:18.271 precision=33.875%
epoch:1, nombre batch:801 loss:18.107 precision=34.469%
epoch:1, nombre batch:901 loss:18.496 precision=33.062%
epoch:1, nombre batch:1001 loss:17.796 precision=35.594%
epoch:1, nombre batch:1101 loss:18.478 precision=33.125%
epoch:1, nombre batch:1201 loss:17.580 precision=36.375%
epoch:1, nombre batch:1301 loss:17.908 precision=35.188%
epoch:1, nombre batch:1401 loss:17.692 precision=35.969%
epoch:1, nombre batch:1501 loss:18.478 precision=33.125%
epoch:1, nombre batch:1601 loss:18.660 precision=32.469%
epoch:1, nombre batch:1701 loss:18.582 precision=32.750%
epoch:1, nombre batch:1801 loss:18.116 p

KeyboardInterrupt: 

In [ ]:
########Test MLP à 2 couches cachées
Tableau_Mlp2=np.arange(20).reshape(crossval_nbr,4)
for section in range(crossval_nbr):
    print("Train {} :\n".format(section))
    Data_Loader.dataset.Set_TestPart_Num(section)
    Train(Mlp2, optimizeMlp2, criterion, Data_Loader)
    print("Test {} :\n".format(section))
    Tableau_Mlp2[section]=Test(Mlp2, criterion, Data_Loader)

Res_Mlp2=mean(Tableau_Mlp2,axis=0)

print("tableau pour les {} étapes de validations:\n".format(crossval_nbr))
print(Tableau_Mlp2)

print("Moyennne des metrics:\n")
print(Res_Mlp2)

In [32]:
Train_Loader.dataset.rate_train=0.9
print(Train_Loader.dataset.rate_train)
print(type(float(int(1))))
#print(torch.tensor(1 if [1,0,0.5,0.4,0.7][i] > 0.5 else 0 for i in range(5)))
#print(torch.tensor([1 if i==2 else 0 for i in range(7)]))

NameError: name 'Train_Loader' is not defined

In [17]:
#TRAIN Perceptron

Epoch=10
#Data_Loader.dataset.Set_TestPart_Num(4)
print(Data_Loader.dataset.debut_test)
log_interval=100
predicted=np.empty(batch_size)
#print(type(Percep.fc1.weight[0][0].item()))
#print(type(Percep.fc1.bias[0].item()))
for epoch in range(Epoch):
    ams=0
    TP=0 #True Positive
    FP=0 # False Positive
    S=0 #True Positive pondéré
    B=0 # False Positive pondéré
    Loss=0
    correct=0
    
    fp=0
    tp=0
    total=0
    running_loss=0
    for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            optimizerPercep.zero_grad()
            
            #print(inputs[0][0].requires_grad)
            #print(labels.size(0))
            #print(inputs[0][0])
            
            
            outputs= Percep(inputs[:,1:31])
            
            #print(outputs[0]+outputs[1])
           # print(outputs.view(-1,32))
        
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= criterion(outputs, labels.float())
            loss.backward()
            optimizerPercep.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                if tp+fp>0:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                else:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} ".format(epoch+1, batch_id+1, running_loss/log_interval)) 

                total=0
                tp=0
                fp=0
                running_loss=0
    if TP+FP>0:
        print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
    else:
        print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}  , AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), AMS(S,B) ))
        
print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f},  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))

        

654588
epoch:1, nombre batch:1 loss:0.078 precision=40.000%
epoch:1, nombre batch:101 loss:8.172 precision=70.538%
epoch:1, nombre batch:201 loss:8.876 precision=58.987%
epoch:1, nombre batch:301 loss:8.327 precision=72.549%
epoch:1, nombre batch:401 loss:8.489 precision=65.000%
epoch:1, nombre batch:501 loss:8.777 precision=67.059%
epoch:1, nombre batch:601 loss:8.415 precision=69.868%
epoch:1, nombre batch:701 loss:8.688 precision=75.966%
epoch:1, nombre batch:801 loss:8.217 precision=76.667%
epoch:1, nombre batch:901 loss:8.200 precision=72.378%
epoch:1, nombre batch:1001 loss:8.476 precision=71.038%
epoch:1, nombre batch:1101 loss:8.279 precision=67.330%
epoch:1, nombre batch:1201 loss:8.692 precision=65.217%
epoch:1, nombre batch:1301 loss:8.394 precision=64.249%
epoch:1, nombre batch:1401 loss:8.757 precision=57.287%
epoch:1, nombre batch:1501 loss:8.604 precision=63.298%
epoch:1, nombre batch:1601 loss:8.422 precision=68.712%
epoch:1, nombre batch:1701 loss:8.663 precision=73.76

epoch:1, nombre batch:14701 loss:9.130 precision=60.531%
epoch:1, nombre batch:14801 loss:9.173 precision=54.029%
epoch:1, nombre batch:14901 loss:9.633 precision=48.611%
epoch:1, nombre batch:15001 loss:9.031 precision=50.061%
epoch:1, nombre batch:15101 loss:8.713 precision=61.124%
epoch:1, nombre batch:15201 loss:8.210 precision=78.853%
epoch:1, nombre batch:15301 loss:8.391 precision=77.295%
epoch:1, nombre batch:15401 loss:8.378 precision=75.986%
epoch:1, nombre batch:15501 loss:8.553 precision=61.136%
epoch:1, nombre batch:15601 loss:8.397 precision=63.124%
epoch:1, nombre batch:15701 loss:8.167 precision=68.000%
epoch:1, nombre batch:15801 loss:8.401 precision=68.219%
epoch:1, nombre batch:15901 loss:8.228 precision=61.298%
epoch:1, nombre batch:16001 loss:8.189 precision=67.062%
epoch:1, nombre batch:16101 loss:8.583 precision=54.893%
epoch:1, nombre batch:16201 loss:8.110 precision=57.635%
epoch:1, nombre batch:16301 loss:9.803 precision=51.444%
epoch:1, nombre batch:16401 los

/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([28])) that is different to the input size (torch.Size([28, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


NameError: name 'Train_Loader' is not defined

In [99]:
torch.save(Percep.state_dict(), 'Percep_weights.pth')
Percep.load_state_dict(torch.load('Percep_weights.pth'))
print(Percep.fc1.state_dict())

<All keys matched successfully>

In [15]:
#TRAIN MLP 1 couche

Epoch=10

log_interval=100
predicted=np.empty(batch_size)
#print(type(Percep.fc1.weight[0][0].item()))
#print(type(Percep.fc1.bias[0].item()))
for epoch in range(Epoch):
    ams=0
    TP=0 #True Positive
    FP=0 # False Positive
    S=0 #True Positive pondéré
    B=0 # False Positive pondéré
    Loss=0
    correct=0
    
    fp=0
    tp=0
    total=0
    running_loss=0
    for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            optimizeMlp1.zero_grad()
            
            #print(inputs[0].requires_grad)
            #print(labels.size(0))
            
            
            
            outputs= Mlp1(inputs[:,1:31])
            
            #print(outputs[0]+outputs[1])
           # print(outputs.view(-1,32))
        
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= criterion(outputs, labels.float())
            loss.backward()
            optimizeMlp1.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                if tp+fp>0:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                total=0
                tp=0
                fp=0
                running_loss=0
    print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}%,  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
   
        
print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f}%,  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Train_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))

        

epoch:1, nombre batch:1501 loss:5.949 precision=0.000%
epoch:1, nombre batch:1601 loss:5.938 precision=59.292%
epoch:1, nombre batch:1701 loss:5.974 precision=58.969%
epoch:1, nombre batch:1801 loss:6.013 precision=63.218%
epoch:1, nombre batch:1901 loss:6.325 precision=63.678%
epoch:1, nombre batch:2001 loss:6.226 precision=61.317%
epoch:1, nombre batch:2101 loss:5.854 precision=59.436%
epoch:1, nombre batch:2201 loss:6.308 precision=64.530%
epoch:1, nombre batch:2301 loss:6.184 precision=69.196%
epoch:1, nombre batch:2401 loss:5.914 precision=62.198%
epoch:1, nombre batch:2501 loss:5.627 precision=53.236%
epoch:1, nombre batch:2601 loss:6.153 precision=61.354%
epoch:1, nombre batch:2701 loss:5.888 precision=67.202%
epoch:1, nombre batch:2801 loss:6.304 precision=61.053%
epoch:1, nombre batch:2901 loss:6.012 precision=60.862%
epoch:1, nombre batch:3001 loss:5.953 precision=63.148%
epoch:1, nombre batch:3101 loss:6.084 precision=59.959%
epoch:1, nombre batch:3201 loss:6.273 precision=6

epoch:1, nombre batch:16201 loss:6.099 precision=62.368%
epoch:1, nombre batch:16301 loss:5.905 precision=61.172%
epoch:1, nombre batch:16401 loss:5.815 precision=52.248%
epoch:1, nombre batch:16501 loss:5.877 precision=64.135%
epoch:1, nombre batch:16601 loss:6.223 precision=62.942%
epoch:1, nombre batch:16701 loss:6.001 precision=62.677%
epoch:1, nombre batch:16801 loss:5.980 precision=64.882%
epoch:1, nombre batch:16901 loss:6.174 precision=64.242%
epoch:1, nombre batch:17001 loss:6.397 precision=67.946%
epoch:1, nombre batch:17101 loss:6.260 precision=63.103%
epoch:1, nombre batch:17201 loss:5.698 precision=64.386%
epoch:1, nombre batch:17301 loss:6.357 precision=62.664%
epoch:1, nombre batch:17401 loss:6.067 precision=61.821%
epoch:1, nombre batch:17501 loss:6.017 precision=67.249%
epoch:1, nombre batch:17601 loss:6.072 precision=59.961%
epoch:1, nombre batch:17701 loss:5.902 precision=60.682%
epoch:1, nombre batch:17801 loss:6.115 precision=56.554%
epoch:1, nombre batch:17901 los

/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch:2, nombre batch:1 loss:0.070 precision=75.000%
epoch:2, nombre batch:101 loss:6.149 precision=67.421%
epoch:2, nombre batch:201 loss:6.348 precision=63.333%
epoch:2, nombre batch:301 loss:5.869 precision=57.114%
epoch:2, nombre batch:401 loss:6.321 precision=60.277%
epoch:2, nombre batch:501 loss:6.334 precision=62.422%
epoch:2, nombre batch:601 loss:6.255 precision=61.555%
epoch:2, nombre batch:701 loss:6.103 precision=60.625%
epoch:2, nombre batch:801 loss:6.013 precision=65.173%
epoch:2, nombre batch:901 loss:5.873 precision=66.736%
epoch:2, nombre batch:1001 loss:6.065 precision=59.341%
epoch:2, nombre batch:1101 loss:5.886 precision=64.858%
epoch:2, nombre batch:1201 loss:6.113 precision=63.043%
epoch:2, nombre batch:1301 loss:5.968 precision=58.584%
epoch:2, nombre batch:1401 loss:5.720 precision=55.754%
epoch:2, nombre batch:1501 loss:5.864 precision=58.758%
epoch:2, nombre batch:1601 loss:5.994 precision=66.061%
epoch:2, nombre batch:1701 loss:6.108 precision=62.109%
epoc

epoch:2, nombre batch:14601 loss:5.909 precision=64.739%
epoch:2, nombre batch:14701 loss:5.780 precision=65.060%
epoch:2, nombre batch:14801 loss:6.271 precision=63.617%
epoch:2, nombre batch:14901 loss:5.909 precision=66.027%
epoch:2, nombre batch:15001 loss:6.400 precision=64.257%
epoch:2, nombre batch:15101 loss:6.028 precision=65.029%
epoch:2, nombre batch:15201 loss:6.185 precision=66.341%
epoch:2, nombre batch:15301 loss:5.996 precision=61.462%
epoch:2, nombre batch:15401 loss:6.056 precision=67.273%
epoch:2, nombre batch:15501 loss:5.921 precision=62.639%
epoch:2, nombre batch:15601 loss:6.154 precision=66.114%
epoch:2, nombre batch:15701 loss:6.119 precision=61.812%
epoch:2, nombre batch:15801 loss:6.040 precision=61.214%
epoch:2, nombre batch:15901 loss:5.817 precision=62.141%
epoch:2, nombre batch:16001 loss:6.208 precision=61.598%
epoch:2, nombre batch:16101 loss:5.989 precision=61.915%
epoch:2, nombre batch:16201 loss:6.133 precision=63.415%
epoch:2, nombre batch:16301 los

...

epoch:8, nombre batch:6601 loss:5.991 precision=61.680%
epoch:8, nombre batch:6701 loss:6.445 precision=66.958%
epoch:8, nombre batch:6801 loss:6.080 precision=62.880%
epoch:8, nombre batch:6901 loss:5.881 precision=66.735%
epoch:8, nombre batch:7001 loss:6.083 precision=65.155%
epoch:8, nombre batch:7101 loss:6.038 precision=63.333%
epoch:8, nombre batch:7201 loss:5.828 precision=67.191%
epoch:8, nombre batch:7301 loss:6.269 precision=66.724%
epoch:8, nombre batch:7401 loss:6.191 precision=64.344%
epoch:8, nombre batch:7501 loss:5.810 precision=61.456%
epoch:8, nombre batch:7601 loss:6.192 precision=60.000%
epoch:8, nombre batch:7701 loss:6.210 precision=64.583%
epoch:8, nombre batch:7801 loss:5.933 precision=63.082%
epoch:8, nombre batch:7901 loss:6.241 precision=58.845%
epoch:8, nombre batch:8001 loss:6.184 precision=63.548%
epoch:8, nombre batch:8101 loss:5.951 precision=64.000%
epoch:8, nombre batch:8201 loss:5.952 precision=65.173%
epoch:8, nombre batch:8301 loss:6.092 precision=

epoch:8, nombre batch:21201 loss:6.349 precision=63.169%
epoch:8, nombre batch:21301 loss:6.163 precision=58.086%
epoch:8, nombre batch:21401 loss:6.190 precision=61.144%
epoch:8, nombre batch:21501 loss:6.166 precision=61.582%
epoch:8, nombre batch:21601 loss:5.828 precision=64.694%
epoch:8, nombre batch:21701 loss:5.987 precision=60.035%
epoch:8, nombre batch:21801 loss:6.115 precision=59.520%
epoch:8, nombre batch:21901 loss:5.996 precision=56.564%
epoch:8, nombre batch:22001 loss:5.934 precision=62.094%
epoch:8, nombre batch:22101 loss:5.887 precision=62.417%
epoch:8, nombre batch:22201 loss:6.043 precision=57.971%
epoch:8, nombre batch:22301 loss:6.089 precision=56.322%
epoch:8, nombre batch:22401 loss:5.880 precision=64.754%
epoch:8, nombre batch:22501 loss:5.989 precision=65.363%
epoch:8, nombre batch:22601 loss:5.879 precision=62.025%
epoch:8, nombre batch:22701 loss:5.769 precision=58.115%
epoch:8, nombre batch:22801 loss:6.021 precision=64.060%
epoch:8, nombre batch:22901 los

epoch:9, nombre batch:12601 loss:6.120 precision=61.131%
epoch:9, nombre batch:12701 loss:6.098 precision=61.080%
epoch:9, nombre batch:12801 loss:5.761 precision=63.011%
epoch:9, nombre batch:12901 loss:6.013 precision=58.755%
epoch:9, nombre batch:13001 loss:6.067 precision=65.033%
epoch:9, nombre batch:13101 loss:6.252 precision=60.239%
epoch:9, nombre batch:13201 loss:6.115 precision=62.322%
epoch:9, nombre batch:13301 loss:6.182 precision=63.009%
epoch:9, nombre batch:13401 loss:6.264 precision=65.839%
epoch:9, nombre batch:13501 loss:6.263 precision=63.843%
epoch:9, nombre batch:13601 loss:5.974 precision=64.394%
epoch:9, nombre batch:13701 loss:6.230 precision=59.687%
epoch:9, nombre batch:13801 loss:6.104 precision=58.447%
epoch:9, nombre batch:13901 loss:6.017 precision=58.845%
epoch:9, nombre batch:14001 loss:6.013 precision=67.120%
epoch:9, nombre batch:14101 loss:6.252 precision=64.300%
epoch:9, nombre batch:14201 loss:5.808 precision=61.382%
epoch:9, nombre batch:14301 los

epoch:10, nombre batch:3801 loss:5.999 precision=61.217%
epoch:10, nombre batch:3901 loss:5.921 precision=64.948%
epoch:10, nombre batch:4001 loss:5.909 precision=63.708%
epoch:10, nombre batch:4101 loss:5.682 precision=60.521%
epoch:10, nombre batch:4201 loss:6.005 precision=65.328%
epoch:10, nombre batch:4301 loss:6.456 precision=63.636%
epoch:10, nombre batch:4401 loss:5.924 precision=60.644%
epoch:10, nombre batch:4501 loss:6.090 precision=67.598%
epoch:10, nombre batch:4601 loss:6.036 precision=62.606%
epoch:10, nombre batch:4701 loss:6.155 precision=60.219%
epoch:10, nombre batch:4801 loss:5.980 precision=61.776%
epoch:10, nombre batch:4901 loss:5.967 precision=68.261%
epoch:10, nombre batch:5001 loss:5.605 precision=63.864%
epoch:10, nombre batch:5101 loss:5.726 precision=64.137%
epoch:10, nombre batch:5201 loss:6.510 precision=65.336%
epoch:10, nombre batch:5301 loss:5.834 precision=62.247%
epoch:10, nombre batch:5401 loss:5.681 precision=66.154%
epoch:10, nombre batch:5501 los

epoch:10, nombre batch:18201 loss:5.785 precision=55.072%
epoch:10, nombre batch:18301 loss:6.137 precision=60.821%
epoch:10, nombre batch:18401 loss:6.161 precision=63.877%
epoch:10, nombre batch:18501 loss:5.897 precision=62.257%
epoch:10, nombre batch:18601 loss:6.205 precision=62.162%
epoch:10, nombre batch:18701 loss:6.178 precision=63.542%
epoch:10, nombre batch:18801 loss:6.093 precision=62.801%
epoch:10, nombre batch:18901 loss:5.976 precision=62.947%
epoch:10, nombre batch:19001 loss:5.859 precision=66.594%
epoch:10, nombre batch:19101 loss:5.999 precision=60.902%
epoch:10, nombre batch:19201 loss:5.857 precision=65.112%
epoch:10, nombre batch:19301 loss:5.987 precision=63.214%
epoch:10, nombre batch:19401 loss:5.590 precision=60.039%
epoch:10, nombre batch:19501 loss:6.029 precision=62.524%
epoch:10, nombre batch:19601 loss:6.190 precision=66.147%
epoch:10, nombre batch:19701 loss:5.832 precision=66.469%
epoch:10, nombre batch:19801 loss:5.856 precision=60.870%
epoch:10, nomb

In [13]:
print(TP)
print(FP)
print(TP/(TP+FP))

0
0


ZeroDivisionError: division by zero

In [12]:
#TRAIN MLP 2 couche

Epoch=10

log_interval=100
predicted=np.empty(batch_size)
#print(type(Percep.fc1.weight[0][0].item()))
#print(type(Percep.fc1.bias[0].item()))
for epoch in range(Epoch):
    ams=0
    TP=0 #True Positive
    FP=0 # False Positive
    S=0 #True Positive pondéré
    B=0 # False Positive pondéré
    Loss=0
    correct=0
    
    fp=0
    tp=0
    total=0
    running_loss=0
    for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            optimizeMlp2.zero_grad()
            
            #print(inputs)
            #print(labels.size(0))
            
            
            
            outputs= Mlp2(inputs[:,1:31])
            
            #print(outputs[0]+outputs[1])
           # print(outputs.view(-1,32))
        
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= criterion(outputs, labels.float())
            loss.backward()
            optimizeMlp2.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                total=0
                tp=0
                fp=0
                running_loss=0
    print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}%,  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
   
        
print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f}%,  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))

        

/home/andrei/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch:1, nombre batch:1 loss:0.046 precision=50.000%
epoch:1, nombre batch:101 loss:14.304 precision=32.933%
epoch:1, nombre batch:201 loss:13.472 precision=37.787%
epoch:1, nombre batch:301 loss:13.849 precision=36.504%
epoch:1, nombre batch:401 loss:13.346 precision=36.445%
epoch:1, nombre batch:501 loss:13.675 precision=37.398%
epoch:1, nombre batch:601 loss:13.911 precision=35.283%
epoch:1, nombre batch:701 loss:13.431 precision=37.949%
epoch:1, nombre batch:801 loss:13.442 precision=37.444%
epoch:1, nombre batch:901 loss:13.772 precision=36.706%
epoch:1, nombre batch:1001 loss:13.895 precision=35.990%
epoch:1, nombre batch:1101 loss:14.226 precision=35.248%
epoch:1, nombre batch:1201 loss:13.702 precision=36.889%
epoch:1, nombre batch:1301 loss:13.944 precision=36.125%
epoch:1, nombre batch:1401 loss:14.245 precision=34.044%
epoch:1, nombre batch:1501 loss:13.963 precision=36.273%
epoch:1, nombre batch:1601 loss:13.787 precision=35.985%
epoch:1, nombre batch:1701 loss:14.125 preci

epoch:1, nombre batch:14401 loss:13.574 precision=36.911%
epoch:1, nombre batch:14501 loss:13.547 precision=37.696%
epoch:1, nombre batch:14601 loss:14.315 precision=35.030%
epoch:1, nombre batch:14701 loss:14.329 precision=33.665%
epoch:1, nombre batch:14801 loss:13.539 precision=37.766%
epoch:1, nombre batch:14901 loss:14.022 precision=36.508%
epoch:1, nombre batch:15001 loss:13.682 precision=36.439%
epoch:1, nombre batch:15101 loss:13.966 precision=35.960%
epoch:1, nombre batch:15201 loss:13.655 precision=36.761%
epoch:1, nombre batch:15301 loss:13.520 precision=37.524%
epoch:1, nombre batch:15401 loss:14.462 precision=34.254%
epoch:1, nombre batch:15501 loss:14.195 precision=35.111%
epoch:1, nombre batch:15601 loss:14.295 precision=35.424%
epoch:1, nombre batch:15701 loss:13.480 precision=37.633%
epoch:1, nombre batch:15801 loss:13.715 precision=36.513%
epoch:1, nombre batch:15901 loss:14.344 precision=35.733%
epoch:1, nombre batch:16001 loss:13.909 precision=35.586%
epoch:1, nombr

/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Resultat Epoch 1;  loss:13.904, Accuracy=42.717%,  precision:36.189%, AMS=0.9860
epoch:2, nombre batch:1 loss:0.148 precision=34.483%
epoch:2, nombre batch:101 loss:14.119 precision=36.106%
epoch:2, nombre batch:201 loss:13.634 precision=36.305%
epoch:2, nombre batch:301 loss:13.780 precision=37.638%
epoch:2, nombre batch:401 loss:13.619 precision=37.893%
epoch:2, nombre batch:501 loss:13.490 precision=38.410%
epoch:2, nombre batch:601 loss:13.916 precision=35.845%
epoch:2, nombre batch:701 loss:14.454 precision=34.755%
epoch:2, nombre batch:801 loss:14.049 precision=36.201%
epoch:2, nombre batch:901 loss:13.595 precision=37.528%
epoch:2, nombre batch:1001 loss:14.076 precision=36.784%
epoch:2, nombre batch:1101 loss:13.705 precision=37.425%
epoch:2, nombre batch:1201 loss:13.858 precision=37.080%
epoch:2, nombre batch:1301 loss:13.799 precision=35.413%
epoch:2, nombre batch:1401 loss:14.253 precision=34.895%
epoch:2, nombre batch:1501 loss:14.106 precision=36.016%
epoch:2, nombre batc

...

epoch:7, nombre batch:12701 loss:14.034 precision=36.138%
epoch:7, nombre batch:12801 loss:13.631 precision=37.387%
epoch:7, nombre batch:12901 loss:14.238 precision=35.006%
epoch:7, nombre batch:13001 loss:13.760 precision=37.243%
epoch:7, nombre batch:13101 loss:13.852 precision=36.836%
epoch:7, nombre batch:13201 loss:13.769 precision=38.004%
epoch:7, nombre batch:13301 loss:14.201 precision=35.334%
epoch:7, nombre batch:13401 loss:14.063 precision=35.920%
epoch:7, nombre batch:13501 loss:14.098 precision=36.394%
epoch:7, nombre batch:13601 loss:13.930 precision=36.844%
epoch:7, nombre batch:13701 loss:13.868 precision=36.826%
epoch:7, nombre batch:13801 loss:13.509 precision=38.178%
epoch:7, nombre batch:13901 loss:13.580 precision=37.371%
epoch:7, nombre batch:14001 loss:14.001 precision=36.600%
epoch:7, nombre batch:14101 loss:13.720 precision=37.401%
epoch:7, nombre batch:14201 loss:13.982 precision=37.121%
epoch:7, nombre batch:14301 loss:13.769 precision=38.169%
epoch:7, nombr

epoch:8, nombre batch:3801 loss:13.880 precision=36.494%
epoch:8, nombre batch:3901 loss:13.200 precision=39.925%
epoch:8, nombre batch:4001 loss:13.874 precision=37.657%
epoch:8, nombre batch:4101 loss:13.870 precision=36.789%
epoch:8, nombre batch:4201 loss:13.722 precision=37.392%
epoch:8, nombre batch:4301 loss:13.772 precision=36.890%
epoch:8, nombre batch:4401 loss:14.272 precision=35.786%
epoch:8, nombre batch:4501 loss:14.272 precision=34.951%
epoch:8, nombre batch:4601 loss:13.648 precision=37.739%
epoch:8, nombre batch:4701 loss:13.942 precision=37.230%
epoch:8, nombre batch:4801 loss:13.841 precision=37.048%
epoch:8, nombre batch:4901 loss:13.608 precision=37.216%
epoch:8, nombre batch:5001 loss:13.609 precision=37.112%
epoch:8, nombre batch:5101 loss:14.216 precision=35.083%
epoch:8, nombre batch:5201 loss:13.646 precision=36.141%
epoch:8, nombre batch:5301 loss:13.751 precision=37.299%
epoch:8, nombre batch:5401 loss:13.754 precision=37.706%
epoch:8, nombre batch:5501 loss

epoch:9, nombre batch:9101 loss:13.784 precision=37.158%
epoch:9, nombre batch:9201 loss:14.286 precision=36.829%
epoch:9, nombre batch:9301 loss:13.737 precision=38.058%
epoch:9, nombre batch:9401 loss:14.527 precision=34.471%
epoch:9, nombre batch:9501 loss:13.522 precision=38.088%
epoch:9, nombre batch:9601 loss:13.691 precision=36.888%
epoch:9, nombre batch:9701 loss:13.712 precision=36.953%
epoch:9, nombre batch:9801 loss:14.032 precision=35.501%
epoch:9, nombre batch:9901 loss:14.040 precision=35.564%
epoch:9, nombre batch:10001 loss:14.598 precision=34.538%
epoch:9, nombre batch:10101 loss:13.998 precision=35.814%
epoch:9, nombre batch:10201 loss:13.869 precision=35.965%
epoch:9, nombre batch:10301 loss:13.591 precision=37.353%
epoch:9, nombre batch:10401 loss:14.131 precision=36.539%
epoch:9, nombre batch:10501 loss:14.027 precision=35.447%
epoch:9, nombre batch:10601 loss:13.747 precision=36.796%
epoch:9, nombre batch:10701 loss:14.152 precision=35.356%
epoch:9, nombre batch:1

epoch:10, nombre batch:101 loss:13.699 precision=37.837%
epoch:10, nombre batch:201 loss:13.740 precision=35.778%
epoch:10, nombre batch:301 loss:13.846 precision=36.259%
epoch:10, nombre batch:401 loss:13.702 precision=37.826%
epoch:10, nombre batch:501 loss:13.519 precision=37.895%
epoch:10, nombre batch:601 loss:13.931 precision=36.500%
epoch:10, nombre batch:701 loss:14.056 precision=35.440%
epoch:10, nombre batch:801 loss:13.482 precision=38.616%
epoch:10, nombre batch:901 loss:13.903 precision=37.328%
epoch:10, nombre batch:1001 loss:13.531 precision=37.732%
epoch:10, nombre batch:1101 loss:13.746 precision=37.686%
epoch:10, nombre batch:1201 loss:14.314 precision=35.443%
epoch:10, nombre batch:1301 loss:13.502 precision=37.403%
epoch:10, nombre batch:1401 loss:14.154 precision=35.036%
epoch:10, nombre batch:1501 loss:13.847 precision=37.402%
epoch:10, nombre batch:1601 loss:13.629 precision=37.148%
epoch:10, nombre batch:1701 loss:13.621 precision=37.684%
epoch:10, nombre batch:

epoch:10, nombre batch:14201 loss:14.060 precision=36.468%
epoch:10, nombre batch:14301 loss:14.211 precision=34.939%
epoch:10, nombre batch:14401 loss:14.036 precision=35.274%
epoch:10, nombre batch:14501 loss:13.931 precision=37.285%
epoch:10, nombre batch:14601 loss:13.653 precision=37.217%
epoch:10, nombre batch:14701 loss:13.572 precision=37.861%
epoch:10, nombre batch:14801 loss:14.092 precision=36.242%
epoch:10, nombre batch:14901 loss:14.356 precision=35.822%
epoch:10, nombre batch:15001 loss:14.027 precision=36.513%
epoch:10, nombre batch:15101 loss:13.771 precision=36.838%
epoch:10, nombre batch:15201 loss:14.316 precision=35.627%
epoch:10, nombre batch:15301 loss:14.236 precision=35.104%
epoch:10, nombre batch:15401 loss:13.852 precision=37.068%
epoch:10, nombre batch:15501 loss:14.234 precision=35.821%
epoch:10, nombre batch:15601 loss:14.237 precision=35.610%
epoch:10, nombre batch:15701 loss:13.671 precision=36.977%
epoch:10, nombre batch:15801 loss:13.884 precision=36.80

In [ ]:
#TRAIN CNN

Epoch=10

log_interval=100
predicted=np.empty(batch_size)
#print(type(Percep.fc1.weight[0][0].item()))
#print(type(Percep.fc1.bias[0].item()))
for epoch in range(Epoch):
    ams=0
    TP=0 #True Positive
    FP=0 # False Positive
    S=0 #True Positive pondéré
    B=0 # False Positive pondéré
    Loss=0
    correct=0
    
    fp=0
    tp=0
    total=0
    running_loss=0
    for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            optimizeCnn.zero_grad()
            
            #print(inputs)
            #print(labels.size(0))
            
            
            
            outputs= Cnn(inputs[:,1:31])
            
            #print(outputs[0]+outputs[1])
           # print(outputs.view(-1,32))
        
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= criterion(outputs, labels.float())
            loss.backward()
            optimizeCnn.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                total=0
                tp=0
                fp=0
                running_loss=0
    print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}%,  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
   
        
print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f}%,  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))

        

In [39]:
#TEST Perceptron

correct= 0
total= 0
test_loss = 0
predicted=np.empty(batch_size)
TP=0
FP=0
S=0
B=0

Data_Loader.dataset.train=False
with torch.no_grad():
    for sample in Data_Loader:
        inputs, labels= sample
        #images=images.view(-1,784)
        outputs= Percep(inputs[:,1:31])
        
        total+= labels.size(0)

        for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        #tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        #fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
        
        test_loss+=criterion(outputs,labels.float())
       
        


test_loss/= len(Data_Loader.dataset)

print("Résultat test: {} bonnes classification sur {} données de test:loss moyen={:.8f} Accuracy={:.3f}%, Precision={:.3f}%, AMS={:.3f}".format(correct,total, test_loss/total,100*correct/total, 100*TP/(TP+FP), AMS(S,B)))

Résultat test: 115013 bonnes classification sur 163650 données de test:loss moyen=0.00000158 Accuracy=70.280%, Precision=66.881%, AMS=0.317


/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
